In [2]:
# =========================
# IMPORTS
# =========================

import boto3
import os
from boto3.s3.transfer import TransferConfig

print("🚀 Starting AWS S3 → RunPod S2 transfer (scalers PKL files)")

# =========================
# CONFIG
# =========================

SOURCE_BUCKET = "ai-bmi-predictor-v2"
SOURCE_PREFIX = "scalers/"

DEST_BUCKET = "e9tcw5eupu"
DEST_PREFIX = "scalers/"

LOCAL_TMP_DIR = "/tmp/scalers"
os.makedirs(LOCAL_TMP_DIR, exist_ok=True)

# =========================
# AWS S3 CLIENT (SOURCE)
# =========================

aws_s3 = boto3.client("s3")

# =========================
# RUNPOD S2 CLIENT (DEST)
# =========================

runpod_s3 = boto3.client(
    "s3",
    aws_access_key_id="user_37sKcYrvnk9UXaIY3B3Zr90MH0g",
    aws_secret_access_key="rps_YW72UMRXEMRVC8A407OCL08J8G34U1B3QTNO1ETX18pa1n",
    endpoint_url="https://s3api-eu-ro-1.runpod.io",
    region_name="eu-ro-1",
)

# =========================
# MULTIPART CONFIG
# =========================

transfer_config = TransferConfig(
    multipart_threshold=10 * 1024 * 1024,   # 10 MB
    multipart_chunksize=10 * 1024 * 1024,
    max_concurrency=2,
    use_threads=True,
)

# =========================
# LIST SOURCE OBJECTS
# =========================

print("📂 Listing PKL files in AWS S3 scalers/")

response = aws_s3.list_objects_v2(
    Bucket=SOURCE_BUCKET,
    Prefix=SOURCE_PREFIX,
)

objects = response.get("Contents", [])

pkl_files = [
    obj["Key"] for obj in objects if obj["Key"].endswith(".pkl")
]

print(f"🔍 Found {len(pkl_files)} PKL files")

# =========================
# TRANSFER LOOP
# =========================

for key in pkl_files:
    filename = os.path.basename(key)
    local_path = os.path.join(LOCAL_TMP_DIR, filename)
    dest_key = f"{DEST_PREFIX}{filename}"

    print(f"\n📥 Downloading: s3://{SOURCE_BUCKET}/{key}")

    aws_s3.download_file(
        Bucket=SOURCE_BUCKET,
        Key=key,
        Filename=local_path,
    )

    size_kb = os.path.getsize(local_path) / 1024
    print(f"✅ Downloaded {filename} ({size_kb:.2f} KB)")

    print(f"⬆️ Uploading to RunPod: s3://{DEST_BUCKET}/{dest_key}")

    runpod_s3.upload_file(
        Filename=local_path,
        Bucket=DEST_BUCKET,
        Key=dest_key,
        Config=transfer_config,
    )

    print(f"🎉 Uploaded: {dest_key}")

print("\n✅ All scaler PKL files transferred successfully!")


🚀 Starting AWS S3 → RunPod S2 transfer (scalers PKL files)
📂 Listing PKL files in AWS S3 scalers/
🔍 Found 3 PKL files

📥 Downloading: s3://ai-bmi-predictor-v2/scalers/scaler_robust_features.pkl
✅ Downloaded scaler_robust_features.pkl (118.36 KB)
⬆️ Uploading to RunPod: s3://e9tcw5eupu/scalers/scaler_robust_features.pkl
🎉 Uploaded: scalers/scaler_robust_features.pkl

📥 Downloading: s3://ai-bmi-predictor-v2/scalers/scaler_standard_features.pkl
✅ Downloaded scaler_standard_features.pkl (0.58 KB)
⬆️ Uploading to RunPod: s3://e9tcw5eupu/scalers/scaler_standard_features.pkl
🎉 Uploaded: scalers/scaler_standard_features.pkl

📥 Downloading: s3://ai-bmi-predictor-v2/scalers/scaler_targets.pkl
✅ Downloaded scaler_targets.pkl (1.02 KB)
⬆️ Uploading to RunPod: s3://e9tcw5eupu/scalers/scaler_targets.pkl
🎉 Uploaded: scalers/scaler_targets.pkl

✅ All scaler PKL files transferred successfully!
